In [1]:
# Parameters
num_cores = 27
mem = 650000
config = "configs.config_Reddit"


In [2]:
#!/usr/bin/env python
# coding: utf-8

if 'config' not in locals():
    config = 'configs.config_SE_test_local'
#     config = 'configs.config_Reddit_test_local'

In [3]:
### initialization ###

from _imports import *
# from _utils import *

print('config file is set to {}'.format(config))

util=importlib.import_module('_utils')
importlib.reload(util)

c=importlib.import_module(config)
importlib.reload(c)

get_ipython().run_line_magic('matplotlib', 'inline')

# param_setup(sys.argv[1:], c)
util.param_setup_ipython(globals(), c)

config file is set to configs.config_Reddit
num_cores is set to 27
mem is set to 650000


In [4]:
### cleaned & tagged text loading + tag file loading ###

df_text_orig = pd.read_csv('{}{}_1_text_original.csv'.format(c.directory['save'], c.project_name))
df_text = pd.read_csv('{}{}_4_text_lemmatized_decoded.csv'.format(c.directory['save'], c.project_name))
pprint(df_text.shape)
util.display_df(df_text.head())

def para_lemmatize(text):
    return [x.strip() for x in text.split(',')]

def para_lemmatize_df(df):
    return df.apply(para_lemmatize)

# c.init_parallel(True)
# df_text['txt_lemmatized']=df_text['txt_lemmatized'].parallel_apply( para_lemmatize)
data_lemmatized = util.parallelize_df(df_text['txt_lemmatized'], para_lemmatize_df, c)

(2225657, 8)


,id,date,category,all_text,group,txt_orig,txt_lemmatized,keywords
0,2qzibe,2015-01-01 00:29:40,MachineLearning,Measuring human performance on standard image ...,submissions,Measuring human performance on standard image ...,"measur, human, zzzltwolfivelonelzzz, standard,...","measur, human, perform, standard, imag classif..."
1,2qzv35,2015-01-01 03:52:22,dataisbeautiful,European economy guide: Taking Europe’s pulse,submissions,European economy guide: Taking Europe’s pulse,"european, economi, guid, take, europ, pul","european, economi, guid, take, europ, pul"
2,2qzzaa,2015-01-01 05:23:25,dataisbeautiful,Popularity of visual forms in DataIsBeautiful ...,submissions,Popularity of visual forms in DataIsBeautiful ...,"popular, zzzlthreelzerolfourlzzz, form, datais...","popular, visual, form, dataisbeauti, base, sam..."
3,2r048c,2015-01-01 07:16:15,dataisbeautiful,JOE GROOMING - DAILY SHAMPOO ~ best shampoo fo...,submissions,JOE GROOMING - DAILY SHAMPOO ~ best shampoo fo...,"joe, groom, daili, shampoo, good, shampoo, oil...","joe, groom, daili, shampoo, good, shampoo, oil..."
4,2r0e7b,2015-01-01 10:25:46,dataisbeautiful,Los Angeles Traffic Accident Rate in Rainy vs ...,submissions,Los Angeles Traffic Accident Rate in Rainy vs ...,"trafficaccid, rate, raini, dri, weather","trafficaccid, rate, raini, dri, weather"


parallel: 27 partitions with 27 cores for para_lemmatize_df


In [5]:
### preping LDA model ###

# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus & Term Document Frequency
def para_corpus(text):
    return id2word.doc2bow(text)

def para_corpus_df(df):
    return df.apply(para_corpus)

corpus=util.parallelize_df(data_lemmatized, para_corpus_df, c)

parallel: 27 partitions with 27 cores for para_corpus_df


In [6]:
### training the optimal LDA model + saving ###

pprint('number of topics = {}, cores = {}, iteration={}'.format(c.num_topics, 
                                                                  c.num_cores, 
                                                                  c.optimal_LDA_iteration_threshold
                                                                 ))

optimal_model = gensim.models.wrappers.LdaMallet(c.directory['mallet'],
                                         corpus=corpus, 
                                         num_topics=c.num_topics, 
                                         id2word=id2word,
                                         workers=c.num_cores,
                                         optimize_interval=1,
                                         iterations=c.optimal_LDA_iteration_threshold,
                                         random_seed=1000
                                        )

'number of topics = 62, cores = 27, iteration=1000'


In [7]:
### saving optimal model + corpus  ###
 
try:
    os.remove('{}optimal_model-{}-{}.db'.format(c.directory['save'],
                                                  c.optimal_LDA_iteration_threshold,
                                                  c.num_topics
                                                  ))
except:
    pass

dbfile = open('{}{}_db_optimal_model-{}-{}.pickle'.format(c.directory['save'],
                                                          c.project_name,
                                                          c.optimal_LDA_iteration_threshold,
                                                          c.num_topics
                                                         ), 'ab')
db=dict(
    model=optimal_model,
    num_topics=c.num_topics,
    df_text=df_text,
    optimal_model_corpus=optimal_model[corpus]
)

# optimal_model.save('{}{}_db_mallet_model-{}-{}.pickle'.format(c.directory['save'],
#                                                               c.project_name,
#                                                               c.optimal_LDA_iteration_threshold,
#                                                               c.num_topics
#                                                              ))
pickle.dump(db, dbfile)
dbfile.close()